In [ ]:
import openai
# openai.api_key = "API_KEY"

# llm_model = "gpt-3.5-turbo"
llm_model = ""

Chat API: OpenAI

In [ ]:
def get_completion(prompt, model = llm_model):
    messages = [{"role":"user", "content":prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0,
    )
    return response.choices[0].message["content"]

get_completion("What is 1+1?")

Chat API: LangChain

In [ ]:
# Define/Format Template & input texts

template_string = """Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```"""

customer_style = """American English in a clam and respectful tone"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

service_style_pirate = """a polite tone that speaks in English Pirate"""
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price, and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

Model

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature = 0.0, model = llm_model)

Prompt template

In [ ]:
from langchain.prompts  import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

customer_message = prompt_template.format_messages(style = customer_style, text = customer_email)
customer_response = chat(customer_message)

print(customer_response.content)

In [ ]:
service_messages = prompt_template.format_messages(style = service_style_pirate, text = service_reply)
service_response = chat(service_messages)

print(service_messages[0].content)
print(service_response.content)

prompt_template.messages[0].prompt ==> full prompt text
prompt_template.messages[0].prompt.input_variables ==> style, text

Output Parser

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
gift_schema = ResponseSchema(name = "gift", description = "Was the item purchased as a gift? Answer True if yes, False if not or unknown.")
delivery_days_schema = ResponseSchema(name = "delivery_days", description="How many days did it take to arrive? If this info is not found, output -1.")
price_value_schema = ResponseSchema(name = "price_value", description="Extract any sentences about the value or price, and output them as a comma seperated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

In [ ]:
prompt = ChatPromptTemplate.from_template(template= review_template_2)

messages = prompt.format_messages(text = customer_review, format_instructions = format_instructions)

response = chat(messages)

output_dict = output_parser.parse(response.content)

In [ ]:
print(type(output_dict))

output_dict.get('delivery_days')